In [ ]:
!pip3 install transformers datasets evaluate torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from huggingface_hub import notebook_login
from transformers import pipeline
import os
import json
import pickle
import numpy as np
import evaluate
import transformers

In [ ]:
id2label = {0: "no", 1: "yes"}
label2id = {"no": 0, "yes": 1}

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [ ]:
# autocast_questions = json.load(open('/Users/vineetraju/Documents/code/kash-is-sped/542/autocast_questions.json'))
# test_questions = json.load(open('/Users/vineetraju/Documents/code/kash-is-sped/542/autocast_competition_test_set.json'))
from google.colab import drive
drive.mount('/content/drive')
notebook_login()
autocast_questions = json.load(open('/content/drive/Shareddrives/Autocast Competition/autocast-master/autocast/autocast_questions.json'))
test_questions = json.load(open('/content/drive/Shareddrives/Autocast Competition/autocast-master/competition/autocast_competition_test_set.json'))
test_ids = [q['id'] for q in test_questions]

In [ ]:
tf_questions = [x for x in autocast_questions if x['id'] not in test_ids and x['qtype'] == 't/f']


In [ ]:
!pip3 install pandas
import pandas as pd

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# convert tf_questions to dataframe
dff = pd.DataFrame(tf_questions)

In [ ]:
dff.head(10)

,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30.138000+00:00,2015-11-30 14:00:15+00:00,[Finance],[http://atimes.com/2015/11/china-will-allow-su...,545,148,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
1,Will the Export-Import Bank of the United Stat...,G5,The Export-Import Bank's authorization expired...,2015-09-01 14:02:21.242000+00:00,2015-12-04 14:00:25+00:00,"[Economic Policy, US Politics, US Policy]",[http://thehill.com/policy/finance/260118-week...,1000,379,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
2,Will a trilateral meeting take place between C...,G6,"A trilateral meeting of leaders from China, Ja...",2015-09-01 14:04:41.470000+00:00,2015-12-31 23:00:11+00:00,[Foreign Policy],"[https://en.wikipedia.org/wiki/Li_Keqiang, htt...",946,385,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
3,Will Iran release Jason Rezaian before 31 Octo...,G7,For details of the case involving Jason Rezaia...,2015-09-01 14:07:22.960000+00:00,2016-01-16 20:00:32+00:00,"[Foreign Policy, Security and Conflict]",[http://www.nytimes.com/2015/10/13/world/middl...,1283,423,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
4,Will North Korea launch a land based missile w...,G8,A launch for military or testing purposes woul...,2015-09-01 14:10:57.372000+00:00,2016-02-06 22:00:38+00:00,[Security and Conflict],[http://www.ctvnews.ca/world/north-korea-carri...,1582,517,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
5,Will the government of Venezuela or Petroleos ...,G9,Plummeting oil prices are threatening the coun...,2015-09-01 14:15:18.926000+00:00,2016-02-29 22:00:06+00:00,[Finance],[http://www.cfr.org/economics/venezuelas-econo...,1010,280,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
6,Will Russia officially lift any part of its fo...,G10,Russia imposed an embargo on food imports from...,2015-09-01 14:18:22.923000+00:00,2015-09-17 06:59:18+00:00,"[Economic Policy, Foreign Policy]",[http://www.popularmechanics.com/space/rockets...,195,138,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
7,Will OPEC announce any changes to its producti...,G12,Changes to the overall quota and the establish...,2015-09-01 14:27:44.197000+00:00,2015-12-31 23:00:43+00:00,"[Economic Policy, Foreign Policy]",[http://in.reuters.com/article/2015/12/02/opec...,1425,542,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
8,"Will the city of Venice lift its ban on ""Jean ...",G13,There is no indication that Venice has lifted ...,2015-09-01 14:32:40.153000+00:00,2016-08-31 17:00:45+00:00,[Entertainment],[https://larswericson.wordpress.com/2016/04/26...,724,151,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
9,Will Africa remain free of the wild poliovirus...,G14,"On August 10, the Global Polio Eradication Ini...",2015-09-01 14:35:18.659000+00:00,2016-08-10 17:00:03+00:00,[Health],[http://www.iflscience.com/health-and-medicine...,1274,328,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."


In [ ]:
def pre_process(df):
	# drop columns
	def drop_columns(df):
		df = df.drop(['tags', 'source_links', 'prediction_count', 'choices', 'status', 'qtype'], axis=1)
		return df
	# drop every row that has answer as 'none'
	def drop_none(df):
		df = df.dropna(subset=['answer'])
		return df
	# explode of crowd into its own row
	def explode_crowd(df):
		df = df.explode('crowd')
		return df
	# extract timestamp from crowd and make it column
	def extract_timestamp(df):
		df['timestamp'] = df['crowd'].apply(lambda x: x['timestamp'])
		return df
	# extract forecast from crowd and make it column
	def extract_forecast(df):
		df['forecast'] = df['crowd'].apply(lambda x: x['forecast'])
		return df
	# remove the crowd column
	def remove_crowd(df):
		df = df.drop(['crowd'], axis=1)
		return df
	# convert timestamps to datetime
	def convert_timestamp(df, col_name):
		df[col_name] = df[col_name].apply(lambda x: x.split('.')[0])
		df[col_name] = pd.to_datetime(df[col_name])
		return df
	# convert yes to 1 and no to 0 in answer column
	def convert_answer(df):
		df['answer'] = df['answer'].apply(lambda x: 1 if x == 'yes' else 0)
		return df
	# rename answer column to labels
	def rename_answer(df):
		df = df.rename(columns={'answer': 'labels'})
		return df
	# lowercase all text
	def lowercase(df):
		df['question'] = df['question'].apply(lambda x: x.lower())
		df['context'] = df['background'].apply(lambda x: x.lower())
		return df



	df = drop_columns(df)
	df = drop_none(df)
	df = explode_crowd(df)
	df = extract_timestamp(df)
	df = extract_forecast(df)
	df = remove_crowd(df)
	df = convert_timestamp(df, "timestamp")
	df = convert_timestamp(df, "publish_time")
	df = convert_timestamp(df, "close_time")
	df = convert_answer(df)
	df = rename_answer(df)
	return df

dff = pre_process(dff)


In [ ]:
dff.head(10)

,question,id,background,publish_time,close_time,forecaster_count,labels,timestamp,forecast
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-01 00:00:00+00:00,0.500
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-01 19:14:09,0.645
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-02 01:33:43,0.425
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-02 01:34:42,0.425
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-02 11:32:27,0.425
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-03 01:07:39,0.275
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-03 06:47:48,0.300
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-03 14:41:32,0.300
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-04 18:15:25,0.300
0,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30,2015-11-30 14:00:15+00:00,148,1,2015-09-04 19:24:19,0.350


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# for each column name passed in, tokenize the column and split it into input_ids and attention_mask for each column
# and then drop the column name

# tokenzize question column
# dff["question_token"] = dff["question"].apply(lambda x: tokenizer(x, padding=True, truncation=True, return_tensors="pt"))

def tokenize_question(text):
    return tokenizer.encode_plus(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')


# Apply the tokenize_question function to the question column and create a new temporary column with dictionaries
dff['tokenized_data'] = dff['question'].apply(tokenize_question)

# Extract input_ids and attention_mask from the dictionaries and create new columns
dff['input_ids'] = dff['tokenized_data'].apply(lambda x: x['input_ids'][0].tolist())
dff['attention_mask'] = dff['tokenized_data'].apply(lambda x: x['attention_mask'][0].tolist())

# Remove the temporary tokenized_data column
dff.drop(columns=['tokenized_data'], inplace=True)

In [ ]:
# dff.drop(columns=['question_token'], inplace=True)

In [ ]:
dff.drop(columns=['question'], inplace=True)

In [ ]:
dff.head(10)

In [ ]:
!pip3 install scikit-learn
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(all_text_summary, test['Score'], test_size=0.2, random_state=0)

# split train and test and the variable to predict is the labels column
X_train, X_test, y_train, y_test = train_test_split(dff[['input_ids', 'id', 'attention_mask', 'publish_time', 'close_time', 'timestamp', 'forecast']], dff['labels'], test_size=0.2, random_state=0)

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="bertV_tf_model",
    learning_rate=3e-5,
    # per_device_train_batch_size=32,
    # per_device_eval_batch_size=32,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.005,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [ ]:
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "no", 1: "yes"}
label2id = {"no": 0, "yes": 1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



trainer.train()